# Mageck Pipeline

Lara Brown

In [15]:
from datetime import datetime
from IPython.display import display, Markdown

todays_date = str(datetime.now().date())

display(Markdown(f'##### Last updated: {todays_date}'))

##### Last updated: 2023-06-01

## Overview

Run [MAGeCK](https://sourceforge.net/p/mageck/wiki/Home/) on read count tables from editing screens in order to quantify the effects of gene knockouts. For example, running MAGeCK on an apoliposecretion screen would indicate which genes impair or increase apolipoprotein secretion.

The MAGeCK outputs can then be run through co-essential GSEA (gene set enrichment analysis).

## Requirements

You will need a conda environment with MAGeCK installed and properly functioning, as well as access to Jupyter lab. Further, you will need a count table with the following characteristics:

* Each row should correspond to a different gene that was knocked out in the screen
* Columns should represent one apolipoprotein whose secretion was measured as part of the screen. Columns should also each correspond to...
    * One of a number of replicates for that apolipoprotein
    * One of (usually 4) bins, where each bin represents quantiles of cells with the strongest/least strong observed phenotypes (eg apolipoprotein secretion/uptake)
    * As an example: APOC3_r2_T20 represents the top 20% of cells in terms of APOC3 secretion, from the second replicate
* There should also be rows for control sgRNAs, identifiable somehow by name

## Data Prep

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Ellie_mageck_count_table_RPM_020723_goodreps.txt", sep="\t")

In [3]:
df.head(5)

,sgrna,gene,AHSG_r1_B20,AHSG_r1_B40,AHSG_r1_T20,AHSG_r1_T40,AHSG_r2_B20,AHSG_r2_B40,AHSG_r2_T20,AHSG_r2_T40,...,APOC3_r2_T20,APOC3_r2_T40,APOC3_r3_B20,APOC3_r3_B40,APOC3_r3_T20,APOC3_r3_T40,APOC3_r4_B20,APOC3_r4_B40,APOC3_r4_T20,APOC3_r4_T40
0,A1CF__1,A1CF,0.000000,0.000000,65.316060,1.023492,107.847608,84.860934,53.609688,177.405595,...,70.181546,91.728545,26.275408,74.672085,31.395158,71.159598,78.472088,171.450615,51.067440,141.227615
1,A1CF__2,A1CF,250.507708,150.713025,46.654328,68.573943,3.081360,79.869114,80.414532,163.976486,...,99.189918,95.315695,77.324773,37.948109,55.655053,54.827887,100.806452,121.068197,94.278352,82.168794
2,AAAS__3,AAAS,1.912273,26.708637,80.030117,68.062198,27.732242,131.035266,78.565922,103.898894,...,77.667577,81.991996,138.884301,94.258206,51.373895,122.487833,63.381302,68.429851,78.565293,84.094625
3,AAAS__4,AAAS,124.297718,103.019030,53.473038,48.615855,0.000000,99.836393,54.533993,37.460146,...,91.703886,69.693196,71.318965,75.896218,107.028948,205.312939,39.236044,0.000000,62.852234,36.590791
4,AAMP__5,AAMP,5.736818,30.524157,59.215109,26.610784,251.130859,93.596619,14.788880,12.722314,...,145.977615,104.539794,69.066788,118.740857,111.310106,54.827887,37.425150,54.894276,99.516038,77.033244


### Converting to reads per million

Normalize the data columnwise so read counts are transformed into reads per million. (If data is already normalized, can skip this step, but also won't change results to repeat it.)

In [4]:
df = df.copy()
df.iloc[:,2:] = df.iloc[:,2:].apply(lambda x: (x/x.sum())*1000000, axis=0)

In [5]:
df.head()

,sgrna,gene,AHSG_r1_B20,AHSG_r1_B40,AHSG_r1_T20,AHSG_r1_T40,AHSG_r2_B20,AHSG_r2_B40,AHSG_r2_T20,AHSG_r2_T40,...,APOC3_r2_T20,APOC3_r2_T40,APOC3_r3_B20,APOC3_r3_B40,APOC3_r3_T20,APOC3_r3_T40,APOC3_r4_B20,APOC3_r4_B40,APOC3_r4_T20,APOC3_r4_T40
0,A1CF__1,A1CF,0.000000,0.000000,65.316060,1.023492,107.847608,84.860934,53.609688,177.405595,...,70.181546,91.728545,26.275408,74.672085,31.395158,71.159598,78.472088,171.450615,51.067440,141.227615
1,A1CF__2,A1CF,250.507708,150.713025,46.654328,68.573943,3.081360,79.869114,80.414532,163.976486,...,99.189918,95.315695,77.324773,37.948109,55.655053,54.827887,100.806452,121.068197,94.278352,82.168794
2,AAAS__3,AAAS,1.912273,26.708637,80.030117,68.062198,27.732242,131.035266,78.565922,103.898894,...,77.667577,81.991996,138.884301,94.258206,51.373895,122.487833,63.381302,68.429851,78.565293,84.094625
3,AAAS__4,AAAS,124.297718,103.019030,53.473038,48.615855,0.000000,99.836393,54.533993,37.460146,...,91.703886,69.693196,71.318965,75.896218,107.028948,205.312939,39.236044,0.000000,62.852234,36.590791
4,AAMP__5,AAMP,5.736818,30.524157,59.215109,26.610784,251.130859,93.596619,14.788879,12.722314,...,145.977615,104.539794,69.066788,118.740857,111.310106,54.827887,37.425150,54.894276,99.516038,77.033244


### Creating weighted sums

We experimented with different ways to combine or choose inputs for MAGeCK and ultimately settled on using a weighted sum of the top samples vs a weighted sum of the bottom.

To calculate, double the top/bottom 20% bin RPMs before adding to next 40% bin RPMs. Calling this weighted top or bottom (WTOP/WBOT).

groupby info: https://pandas.pydata.org/docs/user_guide/groupby.html

regex info: https://docs.python.org/3/library/re.html

In [6]:
replicate = "(^[^_]*_[^_]*_)" # match everything up to second underscore
grouped = df.iloc[:,2:].groupby(df.iloc[:,2:].columns.str.extract(replicate, expand=False), axis=1)

In [7]:
for name,group in grouped:
    t20 = name + "T20"
    t40 = name + "T40"
    b20 = name + "B20"
    b40 = name + "B40"
    df[name + "WTOP"] = 2*group[t20] + group[t40]
    df[name + "WBOT"] = 2*group[b20] + group[b40]
    df[name + "SUMTOP"] = group[t20] + group[t40]
    df[name + "SUMBOT"] = group[b20] + group[b40]

In [8]:
df.head(5)

,sgrna,gene,AHSG_r1_B20,AHSG_r1_B40,AHSG_r1_T20,AHSG_r1_T40,AHSG_r2_B20,AHSG_r2_B40,AHSG_r2_T20,AHSG_r2_T40,...,IGFBP1_r2_SUMTOP,IGFBP1_r2_SUMBOT,IGFBP1_r3_WTOP,IGFBP1_r3_WBOT,IGFBP1_r3_SUMTOP,IGFBP1_r3_SUMBOT,IGFBP1_r4_WTOP,IGFBP1_r4_WBOT,IGFBP1_r4_SUMTOP,IGFBP1_r4_SUMBOT
0,A1CF__1,A1CF,0.000000,0.000000,65.316060,1.023492,107.847608,84.860934,53.609688,177.405595,...,58.536665,195.186389,213.212464,317.546558,142.306747,198.927575,161.671420,268.426947,104.576414,166.985584
1,A1CF__2,A1CF,250.507708,150.713025,46.654328,68.573943,3.081360,79.869114,80.414532,163.976486,...,158.270589,147.328142,209.510771,238.539599,146.946904,139.994597,295.171463,219.174474,200.013120,150.552375
2,AAAS__3,AAAS,1.912273,26.708637,80.030117,68.062198,27.732242,131.035266,78.565922,103.898894,...,127.951606,238.352092,391.013776,441.410512,261.715117,278.993750,232.549995,300.094510,162.767209,237.439551
3,AAAS__4,AAAS,124.297718,103.019030,53.473038,48.615855,0.000000,99.836393,54.533993,37.460146,...,247.283466,83.986718,161.310524,262.914211,119.601279,162.544303,66.921790,64.833695,54.234011,52.899417
4,AAMP__5,AAMP,5.736818,30.524157,59.215109,26.610784,251.130859,93.596619,14.788879,12.722314,...,114.500341,138.412625,313.507561,430.818360,200.892599,286.650673,430.993436,411.338568,310.459534,256.192955


Save this dataframe, with data converted to RPM and with weighted sums already calculated, for use in MAGeCK analysis.

In [9]:
df.to_csv("Ellie_mageck_count_table_RPM_020723_goodreps_WSUM.txt", sep='\t', index=False)

### Save control gRNAs to a text file

We run MAGeCK with control gRNAs specified; to do so, we need a text file listing all of the control gRNA names. In this case, all controls either started with CONTROL_ or MCP_, so we just select all of those and save to a file.

In [12]:
gRNAs = df['sgrna'].tolist()

In [13]:
control_gRNAs = [g for g in gRNAs if g.startswith("CONTROL_") or g.startswith("MCP_")]

In [14]:
with open('Ellie_mageck_control_gRNAs_combined_020723.txt', 'w') as filehandle:
    for c in control_gRNAs:
        filehandle.write(f'{c}\n')

## Mageck instructions

1. Install mageck into your conda environment: `conda install -c bioconda mageck`

2. Make sure you are in the directory where your files are stored and where you want the output to write!

3. Run a version of the following in your terminal. This includes first an analysis for APCO3, then a separate analysis for APOE as the apolipoproteins whose phenotypes are measured. Change parameters as needed for your files:

```mageck test -k rpm_weighted_sum_file.txt -t first_weighted_top_col,second_weighted_top_col,etc -c first_weighted_bot_col,second_weighted_bot_col,etc -n ./mageck_out/phenotype_protein_name/desired_output_filename --control-sgrna control_gRNA_list.txt --sort-criteria pos --norm-method control```

What the options mean:

- -k: the gRNA count table (in this case it’s a RPM table)
- -t: treatment samples (in this case the “Top” samples)
- -c: control samples (in this case the “Bot” samples)
- -n: name of the output files
- --control-sgrna: because we use control gRNAs for normalization we need to supply a list of control gRNAs
- --sort-criteria pos: in the output “xxx.gene_summary.txt” the genes will be sorted by the log fold change [log2(top/Bot)] in decreasing order.
- --norm-method control: use control gRNAs for normalization


### Running Mageck for multiple genes

The for loop below creates the necessary directories for Mageck's output and runs Mageck for each of the four genes specified in this particular read count file.

In [17]:
genes = ["AHSG", "APOA1", "APOC3", "IGFBP1"] 
# genes = ["APOA1", "APOC3"] 

for g in genes:
    top_groups = ",".join([col for col in df.columns if (col.startswith(g) and col.endswith("WTOP"))])
    bot_groups = ",".join([col for col in df.columns if (col.startswith(g) and col.endswith("WBOT"))])
    
    out_dir = f"./mageck_out/{g}"
    out_name = f"./mageck_out/{g}/Ellie_mageck_wsum_{g}_020723"
    # print(out_dir)
    # print(out_name)
    # print(top_groups)
    # print(bot_groups)
    
    mageck_cmd1 = "mageck test -k Ellie_mageck_count_table_RPM_020723_goodreps_WSUM.txt"
    mageck_cmd2 = f"-t {top_groups}"
    mageck_cmd3 = f"-c {bot_groups}"
    mageck_cmd4 = f"-n {out_name}"
    mageck_cmd5 = f"--control-sgrna Ellie_mageck_control_gRNAs_combined_020723.txt --sort-criteria pos --norm-method control"
    
    mageck_cmd = " ".join([mageck_cmd1, mageck_cmd2, mageck_cmd3, mageck_cmd4, mageck_cmd5])
    
    !mkdir -p {out_dir}
    
    print(mageck_cmd)
    
    !{mageck_cmd}


mageck test -k Ellie_mageck_count_table_RPM_020723_goodreps_WSUM.txt -t AHSG_r1_WTOP,AHSG_r2_WTOP,AHSG_r3_WTOP -c AHSG_r1_WBOT,AHSG_r2_WBOT,AHSG_r3_WBOT -n ./mageck_out/AHSG/Ellie_mageck_wsum_AHSG_020723 --control-sgrna Ellie_mageck_control_gRNAs_combined_020723.txt --sort-criteria pos --norm-method control
INFO  @ Thu, 01 Jun 2023 13:33:20: Parameters: /Users/lbb34/miniconda3/envs/mageck-env/bin/mageck test -k Ellie_mageck_count_table_RPM_020723_goodreps_WSUM.txt -t AHSG_r1_WTOP,AHSG_r2_WTOP,AHSG_r3_WTOP -c AHSG_r1_WBOT,AHSG_r2_WBOT,AHSG_r3_WBOT -n ./mageck_out/AHSG/Ellie_mageck_wsum_AHSG_020723 --control-sgrna Ellie_mageck_control_gRNAs_combined_020723.txt --sort-criteria pos --norm-method control 
INFO  @ Thu, 01 Jun 2023 13:33:20: Welcome to MAGeCK v0.5.9.2. Command: test 
INFO  @ Thu, 01 Jun 2023 13:33:20: Loading count table from Ellie_mageck_count_table_RPM_020723_goodreps_WSUM.txt  
INFO  @ Thu, 01 Jun 2023 13:33:20: Processing 1 lines.. 
INFO  @ Thu, 01 Jun 2023 13:33:21: Load

## After running MAGECK

You'll be left with a number of different files, ideally stored in a nice directory depending on how you specified the output. In particular, we're looking for the file(s) of the form `{outputname}.gene_summary.txt`.

The file is sorted by rank, but instead, sort it by lfc.

To run co-essential GSEA on the results, navigate to `run_CE_gsea_loop.ipynb`.